# Identificación del Liker

In [3]:
# Importacion de librerias
import os
import spacy
import pandas as pd
from spacy.matcher import Matcher

# Carag del modelo
spacy.require_cpu() # Temporal mientras se arregla problema con uso de GPU
nlp = spacy.load("es_core_news_lg")

# Guardado de datos en un dataframe

In [6]:
files = os.listdir("Raw_Transcriptions")
files[0]

'1001764369_1_transcription.txt'

In [12]:
files = os.listdir("Raw_Transcriptions")
    # Abre el archivo en modo lectura
with open("Raw_Transcriptions/" + files[0], "r", encoding = "utf-8") as archivo:
    aux_list_df = [] # Lista auxiliar para creacion del df
    # Itera sobre cada línea del archivo
    for linea in archivo:
        # Creacion del dataframe con la transcripcion
        transcript = [linea[:17].strip(), linea[19:29].strip(), linea[31:].strip()]
        aux_list_df.append(transcript)
        
    # Creacion del dataframe con la transcripcion
    transcript_df = pd.DataFrame(aux_list_df, columns = ["Tiempo", "Speaker", "Texto"])

transcript_df

,Tiempo,Speaker,Texto
0,0:00:10 - 0:00:10,SPEAKER_01,Hola
1,0:00:11 - 0:00:13,SPEAKER_00,"Sí, buenas tardes ¿Con Juan Cartagena?"
2,0:00:14 - 0:00:15,SPEAKER_01,Sí
3,0:00:16 - 0:00:18,SPEAKER_00,"Hola, hola ¿Cómo estás?"
4,0:00:19 - 0:00:20,SPEAKER_01,Muy bien
...,...,...,...
80,0:09:43 - 0:09:44,SPEAKER_01,Listo. Listo.
81,0:09:44 - 0:09:50,SPEAKER_00,"Listo. Y dime, yo ya me comunicaría entonces c..."
82,0:09:51 - 0:09:53,SPEAKER_01,Bueno. Listo. Hágale pues.
83,0:09:53 - 0:09:56,SPEAKER_00,"Bueno, Juan. Ustedes recuerden que hablaste co..."


# Liker = Primer Speaker

In [17]:
transcript_df.Speaker == transcript_df.Speaker[0]

0      True
1     False
2      True
3     False
4      True
      ...  
80     True
81    False
82     True
83    False
84     True
Name: Speaker, Length: 85, dtype: bool

In [18]:
import numpy as np
np.where(transcript_df.Speaker == transcript_df.Speaker[0], "Liker", "Cliente")

array(['Liker', 'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente',
       'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente',
       'Liker', 'Cliente', 'Liker', 'Cliente', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Cliente',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker',
       'Cliente', 'Liker', 'Liker', 'Cliente', 'Liker', 'Cliente',
       'Liker', 'Cliente', 'Liker', 'Cliente', 'Liker', 'Cliente',
       'Liker'], dtype='<U7')

In [22]:
import numpy as np
transcript_df["Speaker_Asignado_Opt_1"] = np.where(transcript_df.Speaker == transcript_df.Speaker[0], "Liker", "Cliente")
transcript_df # Opción 1 (La sencilla)

,Tiempo,Speaker,Texto,Speaker_Asignado_Opt_1
0,0:00:10 - 0:00:10,SPEAKER_01,Hola,Liker
1,0:00:11 - 0:00:13,SPEAKER_00,"Sí, buenas tardes ¿Con Juan Cartagena?",Cliente
2,0:00:14 - 0:00:15,SPEAKER_01,Sí,Liker
3,0:00:16 - 0:00:18,SPEAKER_00,"Hola, hola ¿Cómo estás?",Cliente
4,0:00:19 - 0:00:20,SPEAKER_01,Muy bien,Liker
...,...,...,...,...
80,0:09:43 - 0:09:44,SPEAKER_01,Listo. Listo.,Liker
81,0:09:44 - 0:09:50,SPEAKER_00,"Listo. Y dime, yo ya me comunicaría entonces c...",Cliente
82,0:09:51 - 0:09:53,SPEAKER_01,Bueno. Listo. Hágale pues.,Liker
83,0:09:53 - 0:09:56,SPEAKER_00,"Bueno, Juan. Ustedes recuerden que hablaste co...",Cliente


# Liker = Persona que más habla

In [23]:
full_df

NameError: name 'full_df' is not defined